# 卷积层经典网络

In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import init

---
## LeNet
7层，其中2层卷积和2层池化层交替出现，最后输出3层全连接得到整体的效果。

In [2]:
class Lenet(nn.Module) :
    def __init__(self) :
        super().__init__()
        
        layer1 = nn.Sequential()
        layer1.add_module('conv1', nn.Conv2d(1, 6, 3, padding=1))
        layer1.add_module('pool1', nn.MaxPool2d(2, 2))
        self.layer1 = layer1
        
        layer2 = nn.Sequential()
        layer2.add_module('conv2', nn.Conv2d(6, 16, 5))
        layer2.add_module('pool2', nn.MaxPool2d(2, 2))
        self.layer2 = layer2
        
        layer3 = nn.Sequential()
        layer3.add_module('fc1', nn.Linear(400, 120))
        layer3.add_module('fc2', nn.Linear(120, 84))
        layer3.add_module('fc3', nn.Linear(84, 10))
        self.layer3 = layer3
        
    def forward(self, x) :
        x = self.layer1(x)
        x = self.layer2(x)
        x.view(x.size(0), -1)
        x = self.layer3(x)
        return x

---
## AlexNet
相比于LeNet层数更深，同事第一册引入激活层ReLU，在全连接层引入了Dropout层防止过拟合

In [3]:
class AlexNet(nn.Module) :
    def __init__(self, num_classes) :
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPlool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x) :
        x = self.features(x)
        x = x.view(x.size(0), 256 * 256 * 256)
        x = self.classifier(x)
        return x

---
## VGGNet
相比于AlexNet, 它使用更多小的滤波器，层叠很多小的滤波器的感受野盒一个大的滤波器是相同的， 还能减小参数，同时有更深的网络结构

In [4]:
class VGG(nn.Module) :
    def __init__(self, num_classes) :
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, pading=1),
            nn.ReLU(True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_sze=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
        
        self._initialize_weights()
    def forward(self, x) :
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

VGG只是对网络层进行不断叠加，并没有太多创新

---
## GoogleNet的Inception模块
Inception模块设计了一个局部的网络拓扑结构，然后将这些模块堆叠在一起形成一个抽象网络结构。具体来说就是运用几个并行的滤波器对水乳进行卷积盒池化，这些滤波器有不同的感受野，做后将输出的结果按深度拼接在一起形成输出层。

新的Inception模块增加了一些 1 × 1的卷积层来降低输入层的难度，使网络参数减少，从而减少了网络的复杂性。

In [5]:
import torch.nn.functional as F

class BasicConv2d(nn.Module) :
    def __init__(self, in_channels, out_channels, **kwargs) :
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
    
    def forward(self, x) :
        x = self.conv(x)
        x = self.bn(x)
        return x

class Inception(nn.Module) :
    def __init__(self, in_channels, out_channels) :
        super().__init__()
        self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)
        
        self.branch5x5_1 = BasicConv2d(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(48, 64, kernel_size=5, padding=2)        
        
        self.branch3x3db1_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        slef.branch3x3db1_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3db1_3 = BasicConv2d(96, 96, kernel_size=3, padding=1)
        
        self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)
    
    def forward(self, x) :
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3db1 = self.branch3x3db1_1(x)
        branch3x3db1 = self.branch3x3db1_2(branch3x3db1)
        branch3x3db1 = self.branch3x3db1_3(branch3x3db1)
        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        out = [branch1x1, branch5x5, branch3x3db1, branch_pool]
        return torch.cat(out, 1)